In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.cluster import AgglomerativeClustering
import shap
import os

In [91]:
EXCEL_PATH = "Seatbelt_400.xlsx"   # change to your file path; if another file, update
SHEET_NAME = 0
RANDOM_STATE = 42
N_CLUSTERS = 4

In [92]:
df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME)

In [93]:
# Normalize headers: remove punctuation/spaces and lower
df.columns = df.columns.str.strip().str.replace('.', '', regex=False).str.replace(' ', '', regex=False).str.lower()

In [94]:
# Profile & situational columns
profile_cols = [
 'age','education','monthlyincome','currentprofession',
 'q6','q7','q8','q9','q10','q11','q12','q13','q14'
]
time_col = "time"
weather_col = "weather"

In [95]:
# Seatbelt columns
seatbelt_raw = ['q15','q16']  # passenger, driver

In [96]:
# Reverse and binarize seatbelt targets
# Original: 1=Always ... 5=Never. Reversed so higher = safer (we keep this reversed for interpretability)
for col in seatbelt_raw:
    rev_col = col + "_rev"
    df[rev_col] = df[col].map({1:5, 2:4, 3:3, 4:2, 5:1})

# Create binary target: compliant = 1 if reversed >=4 (Always or Most of the time), else 0
# You can change threshold as desired (>=4 maps to Always/Most)
for col in ['q16_rev','q15_rev']:
    bin_col = col.replace('_rev','_bin')
    df[bin_col] = (df[col] >= 4).astype(int)

In [97]:
# Attitude blocks
attitude_cols = [f"q{i}" for i in range(19, 29)]    # Q19 - Q28 (attitude toward seatbelt)
attitude2_cols = [f"q{i}" for i in range(29, 35)]   # Q29 - Q34 (attitude driving)

In [98]:
behaviour_raw = [f"q{i}" for i in range(35, 49)]

In [99]:
# If your sheet uses numeric codes, ensure they are numeric type
for col in seatbelt_raw:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows where seatbelt is NA or value 6
df = df[~df[seatbelt_raw].isin([6]).any(axis=1)]
df = df.dropna(subset=seatbelt_raw)  # require both Q15 and Q16 present for simplicity

In [100]:
print("Rows after filtering NA/6 on seatbelt:", len(df))

Rows after filtering NA/6 on seatbelt: 380


In [101]:
# 4. Reverse and binarize seatbelt targets
# Original: 1=Always ... 5=Never. Reversed so higher = safer (we keep this reversed for interpretability)
for col in seatbelt_raw:
    rev_col = col + "_rev"
    df[rev_col] = df[col].map({1:5, 2:4, 3:3, 4:2, 5:1})

# Create binary target: compliant = 1 if reversed >=4 (Always or Most of the time), else 0
# You can change threshold as desired (>=4 maps to Always/Most)
for col in ['q16_rev','q15_rev']:
    bin_col = col.replace('_rev','_bin')
    df[bin_col] = (df[col] >= 4).astype(int)

In [102]:
# Q18 multi-select (already split: 1 = not selected, 2 = selected)
q18_cols = [c for c in df.columns if c.startswith("q18") and c != "q1811"]

# Recode: 1 -> 0, 2 -> 1, else NA
for c in q18_cols:
    df[c] = df[c].map({1:1, 2:0})

# count number of situational reasons selected
df["q18_count"] = df[q18_cols].sum(axis=1)

In [103]:
# Reverse behavior questions so higher = safer (same mapping)
for col in behaviour_raw:
    if col in df.columns:
        df[col + "_rev"] = pd.to_numeric(df[col], errors='coerce').map({1:5,2:4,3:3,4:2,5:1})

In [104]:
behaviour_rev = [c + "_rev" for c in behaviour_raw if c + "_rev" in df.columns]

In [105]:
# Ensure attitude columns numeric
att_cols_present = [c for c in attitude_cols if c in df.columns]
att2_cols_present = [c for c in attitude2_cols if c in df.columns]

In [106]:
# Profile columns - keep those present
profile_present = [c for c in profile_cols if c in df.columns]
print("Profile present:", profile_present)
print("Behavior_rev present:", behaviour_rev)
print("Attitude present:", att_cols_present + att2_cols_present)

Profile present: ['age', 'education', 'monthlyincome', 'currentprofession', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13', 'q14']
Behavior_rev present: ['q35_rev', 'q36_rev', 'q37_rev', 'q38_rev', 'q39_rev', 'q40_rev', 'q41_rev', 'q42_rev', 'q43_rev', 'q44_rev', 'q45_rev', 'q46_rev', 'q47_rev', 'q48_rev']
Attitude present: ['q19', 'q20', 'q21', 'q22', 'q23', 'q24', 'q25', 'q26', 'q27', 'q28', 'q29', 'q30', 'q31', 'q32', 'q33', 'q34']


In [107]:
features = (
    profile_cols +
    [f"q{i}" for i in range(19,35)] +      # Attitudes
    [c + "_rev" for c in behaviour_raw] +   # Reversed behavior
    q18_cols + ["q18_count", time_col, weather_col]
)

In [108]:
print("Final features selected (sample):", features[:50])

Final features selected (sample): ['age', 'education', 'monthlyincome', 'currentprofession', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q19', 'q20', 'q21', 'q22', 'q23', 'q24', 'q25', 'q26', 'q27', 'q28', 'q29', 'q30', 'q31', 'q32', 'q33', 'q34', 'q35_rev', 'q36_rev', 'q37_rev', 'q38_rev', 'q39_rev', 'q40_rev', 'q41_rev', 'q42_rev', 'q43_rev', 'q44_rev', 'q45_rev', 'q46_rev', 'q47_rev', 'q48_rev', 'q181', 'q182', 'q183', 'q184', 'q185', 'q186', 'q187']


In [109]:
# Drop rows with missing features (or you can impute — here we drop to keep pipeline simple)
df_model = df.dropna(subset=features + ['q16_bin', 'q15_bin']).copy()
print("Rows used for modeling:", len(df_model))

Rows used for modeling: 378


In [110]:
# Preprocess: numeric + categorical handling
X = df_model[features].copy()
y_driver = df_model['q16_bin'].copy()
y_pass = df_model['q15_bin'].copy()

In [111]:
# Separate categorical vs numeric: treat profile columns with small unique values as categorical
cat_cols = [c for c in X.columns if X[c].nunique() <= 8 and X[c].dtype != float and X[c].dtype != int]
# but avoid including numeric ordinal columns that we want as numeric; we will refine:
# explicitly define categorical if any (e.g. q9 trip purpose, q14 location, time/weather)
explicit_cat = []
for candidate in ['q9','q14', time_col, weather_col]:
    if candidate and candidate in X.columns:
        explicit_cat.append(candidate)
# finalize categorical set
cat_cols = list(set(cat_cols + explicit_cat))
num_cols = [c for c in X.columns if c not in cat_cols]

print("Categorical columns:", cat_cols)
print("Numeric columns:", num_cols[:20])

Categorical columns: ['q14', 'weather', 'time', 'q9']
Numeric columns: ['age', 'education', 'monthlyincome', 'currentprofession', 'q6', 'q7', 'q8', 'q10', 'q11', 'q12', 'q13', 'q19', 'q20', 'q21', 'q22', 'q23', 'q24', 'q25', 'q26', 'q27']


In [112]:
# One-hot encode categorical columns
if cat_cols:
    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    X_cat = pd.DataFrame(
        ohe.fit_transform(X[cat_cols]),
        index=X.index,
        columns=ohe.get_feature_names_out(cat_cols)
    )
    X = pd.concat([X.drop(columns=cat_cols), X_cat], axis=1)
else:
    X_cat = pd.DataFrame(index=X.index)

In [113]:
# Numeric matrix (fillna with median)
X_num = X[num_cols].apply(pd.to_numeric, errors='coerce')
X_num = X_num.fillna(X_num.median())

In [114]:
# Combine and scale
X_full = pd.concat([X_num, X_cat], axis=1)
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_full), index=X_full.index, columns=X_full.columns)

In [115]:
# 9. Train Random Forests (driver & passenger)
def train_rf_evaluate(X, y, model_name="driver"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
    rf = RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE, class_weight='balanced')
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    y_prob = rf.predict_proba(X_test)[:,1]
    print(f"\n--- {model_name} model evaluation ---")
    print(classification_report(y_test, y_pred))
    try:
        auc = roc_auc_score(y_test, y_prob)
        print("ROC AUC:", auc)
    except:
        pass
    return rf, X_train, X_test, y_train, y_test

In [116]:
rf_driver, Xtr_d, Xte_d, ytr_d, yte_d = train_rf_evaluate(X_scaled, y_driver, "Driver seatbelt")
rf_pass, Xtr_p, Xte_p, ytr_p, yte_p = train_rf_evaluate(X_scaled, y_pass, "Passenger seatbelt")


--- Driver seatbelt model evaluation ---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.93      1.00      0.97        71

    accuracy                           0.93        76
   macro avg       0.47      0.50      0.48        76
weighted avg       0.87      0.93      0.90        76

ROC AUC: 0.8338028169014085


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- Passenger seatbelt model evaluation ---
              precision    recall  f1-score   support

           0       0.62      0.42      0.50        24
           1       0.77      0.88      0.82        52

    accuracy                           0.74        76
   macro avg       0.70      0.65      0.66        76
weighted avg       0.72      0.74      0.72        76

ROC AUC: 0.7516025641025641


In [117]:
# SHAP explanations (use TreeExplainer)
# Use the driver model as main; you can repeat for passenger
explainer = shap.TreeExplainer(rf_driver)

# Compute SHAP values
shap_values = explainer.shap_values(X_scaled)

# Handle binary classifier SHAP shape correctly
# RandomForest returns [array(class0), array(class1)] for binary tasks
if isinstance(shap_values, list):
    shap_vals_for_class = shap_values[1]   # class=1 → seatbelt compliance
else:
    # Catch case where SHAP gives 3D array [samples, features, classes]
    if shap_values.ndim == 3:
        shap_vals_for_class = shap_values[:, :, 1]
    else:
        shap_vals_for_class = shap_values

# Create SHAP importance summary
shap_summary = pd.DataFrame({
    "feature": X_scaled.columns,
    "mean_abs_shap": np.abs(shap_vals_for_class).mean(axis=0)
}).sort_values("mean_abs_shap", ascending=False)

print("\nTop SHAP features (driver model):\n")
display(shap_summary.head(20))

# Save for reporting
shap_summary.to_csv("shap_feature_importance_driver.csv", index=False)



Top SHAP features (driver model):



,feature,mean_abs_shap
38,q46_rev,0.046487
33,q41_rev,0.027786
51,q18_count,0.024401
39,q47_rev,0.023214
34,q42_rev,0.020367
30,q38_rev,0.020307
35,q43_rev,0.017488
37,q45_rev,0.017041
27,q35_rev,0.015885
29,q37_rev,0.015375


In [118]:
# Set global font to Times New Roman, Size 11
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
import os

#Stop matplotlib font warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

#Try to use Times New Roman if available
font_found = False
for font in fm.findSystemFonts():
    if "Times New Roman" in font:
        fm.fontManager.addfont(font)
        plt.rcParams['font.family'] = 'Times New Roman'
        font_found = True
        break

#Fallback if Times New Roman not available
if not font_found:
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Liberation Serif', 'DejaVu Serif']

plt.rcParams['font.size'] = 11

# Plot
plt.figure(figsize=(4, 2))
shap.summary_plot(shap_vals_for_class, X_scaled, show=False)

plt.tight_layout()
plt.savefig("shap_beeswarm_driver.png", dpi=300, bbox_inches='tight')
plt.close()

In [119]:
# Hierarchical clustering on SHAP vectors

# Ensure SHAP matrix and model DF are aligned
shap_matrix = shap_vals_for_class.copy()  # shape: (n_samples, n_features)
shap_df = pd.DataFrame(shap_matrix, columns=X_scaled.columns).reset_index(drop=True)

# Standardize SHAP matrix for clustering
shap_scaler = StandardScaler()
shap_scaled = shap_scaler.fit_transform(shap_matrix)

In [120]:
# Agglomerative clustering
clusterer = AgglomerativeClustering(n_clusters=N_CLUSTERS, linkage='ward')
clusters = clusterer.fit_predict(shap_scaled)

In [121]:
# Ensure df_model index matches
df_model = pd.DataFrame(X_scaled, columns=X_scaled.columns).reset_index(drop=True)
df_model['cluster'] = clusters
df_model['q16_bin'] = y_driver.reset_index(drop=True)   # make sure target added back

# Add q18_count if exists in original data
if 'q18_count' in df.columns:
    df_model['q18_count'] = df['q18_count'].reset_index(drop=True)

print("Cluster counts:\n", df_model['cluster'].value_counts())

Cluster counts:
 cluster
0    237
3    100
2     21
1     20
Name: count, dtype: int64


In [122]:
# 12. Describe clusters: summary stats
summary_cols = ['q16_bin']
if 'q18_count' in df_model.columns: summary_cols.append('q18_count')

cluster_summary = df_model.groupby('cluster')[summary_cols].mean()
cluster_summary.to_excel("cluster_summary_basic.xlsx")

In [123]:
# SHAP cluster means for top features
top_features = shap_summary['feature'].head(10).tolist()

In [124]:
cluster_shap_means = pd.DataFrame(index=sorted(df_model['cluster'].unique()), columns=top_features)

In [125]:
for c in sorted(df_model['cluster'].unique()):
    idx = df_model.index[df_model['cluster'] == c]
    cluster_shap_means.loc[c] = shap_df.loc[idx, top_features].mean().abs().values

cluster_shap_means.to_excel("cluster_shap_means_top10.xlsx")

In [126]:
print("\nCluster summary (basic):")
print(cluster_summary)
print("\nTop SHAP values per cluster saved → cluster_shap_means_top10.xlsx")


Cluster summary (basic):
          q16_bin  q18_count
cluster                     
0        0.987342   1.075949
1        0.000000   1.900000
2        0.904762   0.952381
3        1.000000   0.660000

Top SHAP values per cluster saved → cluster_shap_means_top10.xlsx


In [127]:
# 13. Assign final labels to clusters based on analysis

label_map = {
    3: "Disciplined Safety-Followers",   # Perfect compliance, no excuses
    0: "Routine Rule-Adherers",          # High compliance, minor situational lapses
    2: "Situational Non-Compliers",      # Usually compliant, break rules in certain conditions
    1: "Young Risk-Takers"               # Low compliance, impulsive behavior
}

cluster_profiles = {}
for c in sorted(df_model['cluster'].unique()):
    row = df_model[df_model['cluster'] == c]
    cluster_profiles[c] = {
        'label': label_map.get(c, f"Cluster {c}"),
        'count': len(row),
        'mean_compliance': row['q16_bin'].mean(),
        'mean_q18': row['q18_count'].mean() if 'q18_count' in row.columns else None
    }

# Attach labels to dataframe
df_model['cluster_label'] = df_model['cluster'].map(label_map)

print("\nFinal Cluster Profiles:")
for k, v in cluster_profiles.items():
    print(f"Cluster {k}: {v}")



Final Cluster Profiles:
Cluster 0: {'label': 'Routine Rule-Adherers', 'count': 237, 'mean_compliance': np.float64(0.9873417721518988), 'mean_q18': np.float64(1.0759493670886076)}
Cluster 1: {'label': 'Young Risk-Takers', 'count': 20, 'mean_compliance': np.float64(0.0), 'mean_q18': np.float64(1.9)}
Cluster 2: {'label': 'Situational Non-Compliers', 'count': 21, 'mean_compliance': np.float64(0.9047619047619048), 'mean_q18': np.float64(0.9523809523809523)}
Cluster 3: {'label': 'Disciplined Safety-Followers', 'count': 100, 'mean_compliance': np.float64(1.0), 'mean_q18': np.float64(0.66)}


In [128]:
# Attach label to dataframe
df_model['cluster_label'] = df_model['cluster'].map({k:v['label'] for k,v in cluster_profiles.items()})

In [129]:
print("Cluster profiles:")
for k,v in cluster_profiles.items(): print(k, v)

Cluster profiles:
0 {'label': 'Routine Rule-Adherers', 'count': 237, 'mean_compliance': np.float64(0.9873417721518988), 'mean_q18': np.float64(1.0759493670886076)}
1 {'label': 'Young Risk-Takers', 'count': 20, 'mean_compliance': np.float64(0.0), 'mean_q18': np.float64(1.9)}
2 {'label': 'Situational Non-Compliers', 'count': 21, 'mean_compliance': np.float64(0.9047619047619048), 'mean_q18': np.float64(0.9523809523809523)}
3 {'label': 'Disciplined Safety-Followers', 'count': 100, 'mean_compliance': np.float64(1.0), 'mean_q18': np.float64(0.66)}


In [130]:
# Visualization: cluster counts & top SHAP per cluster

#Set global font to Times New Roman, size 11
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import warnings

# Remove matplotlib font warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

# Try Times New Roman, otherwise fallback
font_found = False
for font in fm.findSystemFonts():
    if "Times New Roman" in font:
        fm.fontManager.addfont(font)
        plt.rcParams['font.family'] = 'Times New Roman'
        font_found = True
        break

if not font_found:
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Liberation Serif', 'DejaVu Serif']

plt.rcParams['font.size'] = 11

# Plot
plt.figure(figsize=(2, 4))

sns.countplot(
    x='cluster_label',
    data=df_model,
    order=[cluster_profiles[c]['label'] for c in sorted(cluster_profiles.keys())]
)

plt.title("Cluster counts (provisional labels)")
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig("cluster_counts.png", dpi=300, bbox_inches='tight')
plt.close()

In [131]:
cluster_shap_means_plot = cluster_shap_means.copy()
#Set index labels as "Cluster_0", "Cluster_1",....
cluster_shap_means_plot.index = [f"Cluster_{i}" for i in cluster_shap_means_plot.index]
cluster_shap_means_plot.plot(kind='bar', figsize=(10,5))
plt.ylabel("Mean SHAP influence (absolute)")
plt.xlabel("Clusters")
plt.xticks(rotation=0)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig("cluster_shap_top_features.png", dpi=150)
plt.close()

In [132]:
# 15. Save cluster assignments and key fields
out_cols = ['cluster','cluster_label','q16_bin'] + top_features
df_model[out_cols].to_excel("cluster_assignments_and_keyfeatures.xlsx", index=False)

print("Pipeline complete")
print("Saved files:")
print("- cluster_assignments_and_keyfeatures.xlsx")
print("- cluster_summary_basic.xlsx")
print("- cluster_shap_means_top10.xlsx")
print("- cluster_counts.png")
print("- cluster_shap_top_features.png")

Pipeline complete
Saved files:
- cluster_assignments_and_keyfeatures.xlsx
- cluster_summary_basic.xlsx
- cluster_shap_means_top10.xlsx
- cluster_counts.png
- cluster_shap_top_features.png
